# ReID Feature Extraction

In [11]:
import pickle
import numpy as np
import os

from sklearn.manifold import TSNE
import seaborn as sns

### 1. Load ResNet-50
We use a pretrained ResNet-50 network to generate ReID features. More specifically, we collect the output of the second last layer, i.e. the input to the last fully-connected layer, for all cropped boxes and use these lower dimensional features as ReID features. In numbers: we get a feature vector of length 2048 for all cropped boxes.

In [15]:
import torch
import torchvision
from torchvision import transforms
from tqdm import tqdm

resnet = torchvision.models.resnet50(pretrained=True)
f = torch.nn.Sequential(*list(resnet.children())[:-1]) # -2: before avg pooling (vorletzter Layer), -1 before FCL (last layer)
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### 2. Data Preprocessing 
Since all input images need to be of same size we resize each cropped box to a more or less arbitrary size of (672,224). We also normalize the images and transform them into PyTorch tensors. 

In [13]:
preprocess = transforms.Compose([   
    transforms.Resize((672,224)), # (h,w)                    
    #transforms.CenterCrop(224),                
    transforms.ToTensor(),                     
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],                
        std=[0.229, 0.224, 0.225]                  
)])

### 3. Generate the ReID features
For all images we generate ReID features using a ResNet-50. For the sake of convenience, we store these features camera-wise in one pickle file. This file type is used since we create dictionaries that connect the ReId features, person ids, frame ids, labels, and camera ids to each other. The result is one pickle file per camera.

In [16]:
cameras = ['C1','C2','C3','C4','C5','C6','C7']

parent_dir = '/home/ge93qew/WILDTRACK/Image_subsets/'
img_child_dir = 'croppedBB/'
store_dir = 'reid-features/' 

for cam in cameras:
    img_path = os.path.join(parent_dir,cam,img_child_dir)
    store_path = os.path.join(parent_dir,cam,store_dir)
    print(f'im_path: {img_path} \nst_path: {store_path}')
    
    features = []
    person_IDs = []
    frame_IDs = []
    cam_IDs = []
    names = []
    
    labels = sorted(os.listdir(img_path))
    for label in tqdm(labels):
        frame_ID, cam_ID, person_ID, name = label.split('-')
        
        person_IDs.append(person_ID)
        frame_IDs.append(frame_ID)
        names.append(name)
        cam_IDs.append(cam_ID)
        
        img = Image.open(os.path.join(img_path,label))
        
        input_tensor = preprocess(img)
        input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
        
        if torch.cuda.is_available():
            input_batch = input_batch.to('cuda')
            f.to('cuda')
            
        feature = f(input_batch).squeeze(0)
        feature = feature.cpu().detach().numpy()
        feature = np.reshape(feature, -1) # -1 indicates that the length of the array is used as dimension => equal to len(b)
        features.append(feature)
        
    dictionary = {z[0]: list(z[1:]) for z in zip(
        range(len(labels)), features, person_IDs, frame_IDs, names, cam_IDs)}
        
    #with open(f'{store_path}/saved_dictionary-{cam}.pkl', 'wb') as file:
    #    pickle.dump(dictionary, file)

im_path: /home/ge93qew/WILDTRACK/Image_subsets/C1/croppedBB/ 
st_path: /home/ge93qew/WILDTRACK/Image_subsets/C1/reid-features/


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8305/8305 [01:44<00:00, 79.37it/s]


im_path: /home/ge93qew/WILDTRACK/Image_subsets/C2/croppedBB/ 
st_path: /home/ge93qew/WILDTRACK/Image_subsets/C2/reid-features/


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3499/3499 [00:46<00:00, 75.19it/s]


im_path: /home/ge93qew/WILDTRACK/Image_subsets/C3/croppedBB/ 
st_path: /home/ge93qew/WILDTRACK/Image_subsets/C3/reid-features/


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3562/3562 [00:48<00:00, 73.06it/s]


im_path: /home/ge93qew/WILDTRACK/Image_subsets/C6/croppedBB/ 
st_path: /home/ge93qew/WILDTRACK/Image_subsets/C6/reid-features/


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8924/8924 [01:48<00:00, 81.91it/s]


im_path: /home/ge93qew/WILDTRACK/Image_subsets/C7/croppedBB/ 
st_path: /home/ge93qew/WILDTRACK/Image_subsets/C7/reid-features/


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3188/3188 [00:42<00:00, 75.04it/s]
